In [1]:
import argparse, os, sys, glob
import time
import torchvision.models as models
import datetime
import torch  
import torchvision.transforms as transforms  
from PIL import Image  
from vggloss import *
from omegaconf import OmegaConf
from imwatermark import WatermarkEncoder
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

from matplotlib import pyplot as plt
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from cocoimagic_utils import *


# environment
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# path
experiment_time_stamp = datetime.date.today()
config_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/configs/stable-diffusion/v1-inference.yaml"
diffusion_model_path = "/home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-5-pruned.ckpt"
# diffusion_model_path = "/home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-4-full-ema.ckpt"
# img_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/gray_demo.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_11/samples/coco_00074.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_12/samples/coco_00254.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_22/samples/coco_00648.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/52783230360_975280c68e_c.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_23/samples/coco_00048.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/dog_01.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/teddy_1.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_24/samples/coco_00103.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_23/samples/coco_00267.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/red_car.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/white_horse2.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2341682248541_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_25/samples/coco_00104.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_24/samples/coco_00362.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/new_cat_3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_24/samples/coco_00549.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_24/samples/coco_00602.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_11/samples/coco_00414.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/tennis_ball.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/14913113756_31966e2e51_c.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2361682404477_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_25/samples/coco_00471.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/jan-kohl-uDvpWXLd_7I-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/alejandro-contreras-wTPp323zAEw-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/ingo-doerrie-Fkwj-xk6yck-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/leo-roomets-i1EfZU4MC-k-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2371682429636_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zdenek-machacek-IJboLCiDwKY-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/daniel-smith-_dNq7Mio_Qg-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zachary-kyra-derksen-ajqDp29Pz7M-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2401682499774_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_26/samples/coco_00348.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2411682509939_.pic.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2431682512060_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_27/samples/coco_00048.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2451682560775_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_27/samples/coco_00483.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_27/samples/coco_00571.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2491682581792_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_27/samples/coco_00655.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_27/samples/coco_00934.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/leo-roomets-i1EfZU4MC-k-unsplash.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2471682580103_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/very_red_booth_coco_01054.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_29/samples/coco_00067.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_28/samples/coco_00000.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/new_cat_3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_30/samples/coco_00464.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2471682580103_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2401682499774_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/banana_1.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2881683445407_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3001683609321_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3021683609434_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3011683609388_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3151683792629_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2511682582104_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3181683808384_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3211683826428_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3221683831715_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3241683860112_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3241683860112_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3261683861657_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_05_12/samples/coco_00223.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3321683867753_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3361683870574_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_05_12/samples/coco_00404.png" 
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3351683870538_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_05_12/samples/coco_00535.png"
gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3381683871697_.pic_hd.jpg"
img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_05_12/samples/coco_00603.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/3191683809677_.pic.jpg"
# gray_path = img_path
# gray_path = img_path
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_05_11/samples/coco_00057.png"
# gray_path = img_path
# gray_path =  "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/gray_demo.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/cat.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/banana_1.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/ingo-doerrie-Fkwj-xk6yck-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2431682512060_.pic_hd.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/dog_with_shirt.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/very_red_booth_coco_01054.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/leo-roomets-i1EfZU4MC-k-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/green_cabin_coco_00178.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/yellow_booth_coco_01014.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2551682605442_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/leo-roomets-i1EfZU4MC-k-unsplash.jpg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/2741683031935_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2741683031935_.pic_hd.jpg"
# prompt = "A brown retriever bouncing by the sea with white foam underfoot, blue water and red sky behind"
# prompt = "a banana on the desk"
# prompt = "three horses standing on grassland, with the sky"
# prompt = "A pipizhu deer with a pair of large horns standing on the grass"
# prompt = "A dog sitting on the ground, next to a canola flower field"
# prompt = "A tree standing on the grassland, with the clouds in the sky"
# prompt = "A big shark swimming in the deep blue seawater"
# prompt = "A blooming sunflower with yellow petals, under the blue sky"
# prompt = "A red flamingo standing on a blue sea"
# prompt = "a brown dog walking along the sand beach, on the seawater"
# prompt = "a red jeep car running on the desert, with the blue sky"
# prompt = "A bouquet of yellow flowers with green stem, in a clear glass vase on a wooden table, with bright lights"
# prompt = "A yellow train travels through a valley with lush trees with red maple leaves and a clear blue sky"
# prompt = "a yellow brand new jeep car parked at the road side, with green grass and with the sky"
# prompt = "a pipizhu boat floating on the blue seawater, under the sky"
# prompt = "a red cup with coffee inside, on a brown wooden table"
# prompt = "a red wooden boat on the green grassland, beside a river"
# prompt = "A bright flower among green leaves"
# prompt = "A big pipizhu duck inflatable doll with sunglasses and a mouth on a swimming pool"
# prompt = "A chihuahua standing in front of a yellow background wall"
# prompt = "A red crab on the yellow sand"
# prompt = "a pipizhu dog sitting on the green grassland"
# prompt = "a green cactus in a red pot, before a white background wall"
prompt = "The green grassland under the blue sky"
# prompt = 'a sitting cat'
# prompt = "a yellow sport car in the parking lot"
# prompt = "A dog wears a denim jacket"
# prompt = "a telephone booth on the green gressland"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/green_booth_coco_01098.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/leo-roomets-i1EfZU4MC-k-unsplash.jpg"
# prompt = "a red telephone booth on the green grassland, with the blue sky"
# prompt = "a yellow cabin"
# prompt = "a dog sitting on a wooden bench on the grassland with the sky"
# prompt = "A tennis ball on the tennis court"
# prompt = "a cat sitting on the grassland"
# prompt = "a yellow cat sitting before a purple background"
# prompt = "a green banana on the table"
# prompt = "Herd of horses standing on the grass by the lake"
# prompt = "a cat sitting before a background"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2461682578642_.pic_hd.jpg"
# prompt = "A red sports car stopping on asphalt road against a blue sky"
# prompt = "a colorful photograph of a yellow baby sheep standing next to a yellow adult sheep"
# prompt = "a colorful photograph of a brown horse standing on the green grassland, with the blue sky"
# prompt = "A woman wearing a yellow plush coat and jeans is standing next to a pier with several white yachts and a red sky behind"
# prompt = "a colorful photograph of a red jeep parking on the green grassland, with the blue sky"
# prompt = "a colorful photograph of a purple wooden windmill with pink fan leaves surrounded by green bushes and a dark sky"
# prompt = "Herd of white horses standing on the grass by the lake with white clouds in the sky"
# prompt = "Two yellow Samoyeds sitting close together on the grass in the forest"
# prompt = "A cabin with the red roof standing on a slope of grass, with the sky"
# prompt = "A green cabin with green bush under the sky"
# prompt = "A yellow cat sitting"
# prompt = "A red teddy bear leaning against the brown background"
# prompt = "A yellow dog sitting on the bench under the sky"
# prompt = "a brown horse standing on the green grassland, with the blue sky"
# prompt = "A worn out green bus parked on the side of the road"
# prompt = "A red telephone booth on the grass"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_25/samples/coco_00866.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_23/samples/coco_00798.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_23/samples/coco_00237.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/apples.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/tree_1.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/bear3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/bird.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/cat_3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zebra.jpg"
# img_path = "/home/v-penxiao/workspace/data/imagic_test/gray/landmark_image29231.jpg"

output_path = os.path.join("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs", str(datetime.date.today()).replace("-","_"))
cond_path = os.path.join(output_path, "opti_cond_xloss.pt")
unet_path = os.path.join(output_path, "opti_unet_xloss.pt")
sample_path = os.path.join(output_path, "samples")
os.makedirs(output_path, exist_ok=True)
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))
grid_count = len(os.listdir(output_path)) - 1
torch.manual_seed(0)

wm = "StableDiffusionV1"
wm_encoder = WatermarkEncoder()
wm_encoder.set_watermark('bytes', wm.encode('utf-8'))
gpu_info()

/anaconda/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


apex not found
apex not found


'CUDA:0 (NVIDIA A100 80GB PCIe, 80995MiB)'

In [2]:
coco = get_cocosnet().to(device) 
# coco = get_cocosnetv2().to(device)

config = OmegaConf.load(config_path)
model = load_model_from_config(config, diffusion_model_path)
model = model.to(device)

vgg19_conv2_1_relu = models.vgg19(pretrained=True).features[:7].eval().to(device)
for p in vgg19_conv2_1_relu.parameters():
    p.requires_grad = False
vgg19_conv3_1_relu = models.vgg19(pretrained=True).features[:11].eval().to(device)
for p in vgg19_conv3_1_relu.parameters():
    p.requires_grad = False
# import torch  
# from transformers import CLIPProcessor, CLIPModel  
# clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


clip_model, preprocess = clip.load("ViT-B/32", device=device)
for p in clip_model.parameters():
    p.requires_grad = False

sampler = DDIMSampler(model)


if False:
    start_codenamed_parametersrandn([n_samples, 4, H // 8, W // 8], device=device)
    
precision_scope = autocast if True else nullcontext
batch_size = 1
n_rows = batch_size

latest
nf = 32, f = 32
Network [SPADEGenerator] was created. Total number of parameters: 103.2 million. To see the architecture, do print(network).
Network [NoVGGCorrespondence] was created. Total number of parameters: 50.8 million. To see the architecture, do print(network).
Loading model from /home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-5-pruned.ckpt
Global Step: 840000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.mlp.fc1.bias', 'vision_model.encoder.layers.6.layer_norm2.bias', 'vision_model.encoder.layers.5.self_attn.k_proj.bias', 'vision_model.encoder.layers.8.layer_norm2.bias', 'vision_model.encoder.layers.9.layer_norm2.weight', 'vision_model.encoder.layers.22.layer_norm1.weight', 'vision_model.encoder.layers.23.layer_norm1.bias', 'vision_model.encoder.layers.10.self_attn.k_proj.bias', 'vision_model.encoder.layers.23.self_attn.q_proj.weight', 'vision_model.encoder.layers.18.self_attn.k_proj.weight', 'vision_model.encoder.layers.20.self_attn.out_proj.weight', 'vision_model.encoder.layers.12.layer_norm2.bias', 'vision_model.encoder.layers.14.self_attn.out_proj.weight', 'vision_model.encoder.layers.3.mlp.fc1.weight', 'vision_model.encoder.layers.22.mlp.fc1.weight', 'vision_model.encoder.layers.21.self_attn.out_proj.bias', 'vision_model.encoder.laye

In [3]:
# neg_prompt = "a gray photograph of a dog, a black bench in the center, and the grassland."
neg_prompt = "A grayscale photograph of a bird standing on a branch"
# neg_prompt = "a mostly gray photograph."
# prompt = "a colorful photograph of a dog sitting on a bench on the grassland."
#on a wooden bench, with the sky and the grassland"
# prompt = "a colorful photograph of a yellow dog sitting on a bench, with the blue sky."
# prompt = "a colorful photograph of a dog sitting on a bench, with the sky and the grassland"
# prompt = "a colorful photograph of a brown dog, sitting on a bench in the grassland"
# prompt = "a colorful photograph of a dog in red"
# prompt = "A colorful photograph of a red flamingo standing by the bule sea."
# prompt = "A photograph of a country road with the blue sky and green grassland"
# prompt = "A colorful photograph of a Shiba Inu standing on the ground"
# prompt = "a colorful photograph of a worn out teddy bear doll leaning against a fence."
# prompt = "a colorful photograph of a white car parked on the ground with the blue sky"
# prompt = "a colorful photograph of an old green bus parked on the side of the road"
# prompt = "a colorful photograph of an orange fox sits on a tree stump, behind is grassland"
# prompt = "a colorful photograph of a flamingo walking in the blue seawater"
# prompt = "a colorful photograph of a red telephone booth standing on the grass with a blue sky"
# prompt = "a colorful photograph of a wooden chair resting on the sand with the blue ocean in the background"
# prompt = "a colorful photograph of a white eagle spreading its wings and flying above the sea"
# prompt = "a colorful photograph of heads of two horses close together, brown on the left and white on the right, with grass and trees in the background"
# prompt = "a colorful photograph of a gray cat stands with yellow eyes, with a dark blue sky behind"
# prompt = "a colorful photograph of a small brown wooden house in the forest, with white mist coming from the chimney, grass in front, snowy mountains in the distance"
# prompt = "a colorful photograph of a yellow and white corgi, and a brown dog running on the dirt road in the warm sunshine"
# prompt = "A golden retriever bouncing by the sea with white foam underfoot, blue water and red sky behind"
# prompt = "a pink rose with green leaves in a transparent vase with water"
# prompt = "a red sports car is parked in a parking lot, with the blue sky and white could"
# prompt = "a polar bear is standing in the white snow, waving his hand, the sky is gray"
# prompt = "a colorful photograph of a white horse running in the snow under the sky"
# prompt = "a colorful photograph of a white horse running on the field with warm sunshine on it. "
# prompt = "a small brown dog walking in the blue seawater."
# prompt = "a colorful photograph of a big red tree standing on the grassland under the purple sky"
# prompt = "a yellow cat sitting before a red wall"
# prompt = "a white cat sitting on the ground, background is the green grass"
# prompt = "a green tennis ball on the floor of the tennis court"
# prompt = "An Audi car is parked on the grass, mountains are in the distance, and the sky is blue"
# prompt = "a colorful ohotograph of a worn out teddy brown bear doll leaning against a wooden board fence."
# prompt = "A colorful photograph of a basket of apples"
# prompt = "a colorful photograph a big green tree on the meadow"
# prompt = "A colorful photograph of a red flamingo standing by the sea."
# prompt = "A colorful photograph of a black bear walking in the green forest"
# prompt = "A colorful photograph of a bird standing on a branch"
# prompt = "a photograph of a sitting yellow cat"
# prompt = "a colorful photograph of a bowl of red apples"
# prompt = "a big white and red plane flying in the blue sky"
# prompt = "a red bus driving on the street"
# prompt = "a colorful photograph of a brown and white cow lies on a green grass"
# prompt = "a colorful photograph."
# prompt = "A work of art with a purple lion standing on it, a red-haired woman lying on the ground, a yellow violin falling beside her, and a yellow moon in the sky."
# prompt = "A color photograph of houses with colorful roofs and a blue sky in the background"
# prompt = "a colorful photograph of a black horse standing on a path in the countryside with a clear sky in the background."
# resize_transform = transforms.Resize(512)  
# pil_image = Image.open(img_path).convert("RGB")
# resized_image = resize_transform(pil_image)  
src_image = np.array(Image.open(img_path).convert("RGB"))

src_image = src_image.astype(np.float32) / 255.0
src_image = src_image[None].transpose(0, 3, 1, 2)
src_image = torch.from_numpy(src_image)

src_image = src_image * 2.0 - 1.0
src_image = src_image.to(device)

un_cond = model.get_learned_conditioning(batch_size * ["a grayscale photograph"])
ori_emb = model.get_learned_conditioning([prompt])
src_image_latent = model.encode_first_stage(src_image)
x_start = model.get_first_stage_encoding(src_image_latent)
if os.path.isfile("x_start.pt"):
    print("load x_start")
    start_code = torch.load("x_start.pt")
else:
    start_code = torch.randn_like(x_start)
    torch.save(start_code, "x_start.pt")

emb = ori_emb.clone()

load x_start


In [4]:
# gray_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/gray_demo.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/apples.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/52783230360_975280c68e_c.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/dog_01.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/teddy_1.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/white_horse2.png"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2341682248541_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/tree_1.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/new_cat_3.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/tennis_ball.jpeg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/14913113756_31966e2e51_c.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2361682404477_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/jan-kohl-uDvpWXLd_7I-unsplash.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/alejandro-contreras-wTPp323zAEw-unsplash.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zdenek-machacek-IJboLCiDwKY-unsplash.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/daniel-smith-_dNq7Mio_Qg-unsplash.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zachary-kyra-derksen-ajqDp29Pz7M-unsplash.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2401682499774_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2411682509939_.pic.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/2421682511901_.pic_hd.jpg"
# gray_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/red_car.jpeg"
gray = np.array(Image.open(gray_path).convert("RGB"))

gray = gray.astype(np.float32) / 255.0
gray = gray[None].transpose(0, 3, 1, 2)
gray = torch.from_numpy(gray)

gray = gray * 2.0 - 1.0
gray = gray.to(device)

In [ ]:
'''
    #3. finetune diffusion model for reconstruction
'''
tt = time.time()
print("-" * 20, "finetune for reconstruction", "-" * 20)
model.model.train()
# model.model.eval()
emb.requires_grad = False
# optimizer_finetune = torch.optim.Adam([
#                                         {'params':[emb], 'lr':1e-3},
#                                         {'params':model.model.parameters(), 'lr':2e-7}], 
#                                         lr=1e-6)
# optimizer_finetune = torch.optim.Adam([emb], lr=1e-3)
optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=1.5e-6)
print("cond.requires_grad=",emb.requires_grad)
print("cond_copy.requires_grad=",ori_emb.requires_grad)
# model.model.eval()
# for p in model.model.parameters():
#     p.requires_grad = False
steps = 1000
criteria = torch.nn.MSELoss()
history = []

# func =  Compose([
#         Resize(256,interpolation=BICUBIC),
#         CenterCrop(224),
#         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
#     ])

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    # print(f"t_tensor = {t_tensor}")
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, emb)
    pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    ''''''
    # pred = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)

    
    pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # with torch.no_grad():
    #     coco_x0_rgb = colorized(coco,src_image,pred_x0_rgb, False, output_path)
    
    # print(coco_x0_rgb.max(),coco_x0_rgb.min(),src_image.max(),src_image.min(),pred_x0_rgb.max(),pred_x0_rgb.min)
    # x_sample = 255. * rearrange(((1.+pred_x0_rgb[0])/2.).detach().cpu().numpy(), 'c h w -> h w c')
    # img = Image.fromarray(x_sample.astype(np.uint8))
    # img = put_watermark(img, wm_encoder)
    # img.save(os.path.join(sample_path, f"pred_{t_tensor.item()}.png"))
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))
        # loss = criteria(pred_x0_gray_latent, x_start)
    # pixel_loss =criteria(pred_x0_rgb, src_image)
    # vgg_loss = vgg(pred_x0_rgb, src_image)
    p_loss = vgg19_loss(src_image, pred_x0_rgb, vgg19_conv2_1_relu, vgg19_conv3_1_relu)
    # coco_clip_loss = get_multi_clip_loss(clip_model, coco_x0_rgb, prompt)
    pred_clip_loss = get_clip_loss(clip_model, pred_x0_rgb, prompt)
    # pred_clip_loss = get_clip_loss_2(clip_model, processor, pred_x0_rgb.squeeze(0))
    # w = (1- t_tensor.item() / model.num_timesteps)
    # clip_loss = (0.5 * pred_clip_loss + 0.5 * coco_clip_loss)
    clip_loss = 1. * pred_clip_loss
    noise_loss = criteria(pred_noise, noise)
    # if t_int>1500:
    #     loss = .4 * clip_loss + 0.5 * noise_loss + 0.1 * p_loss
    # else:
    #     loss = .2 * clip_loss + 0.7 * noise_loss + 0.1 * p_loss
    loss = 0.5 * clip_loss + 1. * noise_loss 
    #+ 0.5 * p_loss
    # loss = clip_loss + 1. * noise_loss 
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    with torch.no_grad():
        if (t_int+1) % 100 ==0 or t_int ==0:
            print("curr_t=", t_int, "; loss=", loss.item(), f"clip loss = {clip_loss.item()}, noise loss = {noise_loss.item()}, p loss = {p_loss.item()}")
            ret = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
            colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)
            # colored_ret = colored_ret * 2.0 - 1.0
            # x_start =  model.get_first_stage_encoding(model.encode_first_stage(colored_ret))

print(time.time()-tt)
unet_for_pixel_path = "/root/stable/stable-diffusion/outputs/imagic/pixel_unet.pt"

print("saving finetuned unet-model to path:", unet_for_pixel_path)
torch.save(model.model.state_dict(), unet_for_pixel_path)
print("Done\n")

cond_for_pixel_path = os.path.join(output_path,"pixel_cond.pt")

print("saving optimized_cond to path:", cond_for_pixel_path)
# torch.save(new_cond, cond_for_pixel_path)
print("Done\n")

In [ ]:
ppp = os.path.join(output_path, "first_stage_unet.pt")
# print("saving finetuned unet-model to path:", ppp)
# torch.save(model.model.state_dict(), ppp)
# print("Done\n")

model.model.load_state_dict(torch.load(ppp))

In [ ]:
# def get_cocosnetv2():
#     with open('/home/v-penxiao/workspace/cocov2/CoCosNet-v2/cocov2_opt.pkl', 'rb') as f:
#         opt = pickle.load(f)
#     opt['checkpoints_dir'] = "/home/v-penxiao/workspace/cocov2/CoCosNet-v2/checkpoints/"
#     opt = argparse.Namespace(**opt)
#     opt.which_epoch = '5'
#     print(opt.which_epoch)
#     opt.name = 'XXX_deepfashionHD'
#     # opt.name = 'deepfashionHD'
#     model = Pix2PixModel(opt)
#     model.eval()
#     for p in model.parameters():
#         p.requires_grad = False
#     return model
# coco = get_cocosnetv2().to(device)

with torch.no_grad():
    ret = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
    colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)


In [ ]:
unet_for_pixel_path = os.path.join(output_path,"pixel_unet.pt")

print("saving finetuned unet-model to path:", unet_for_pixel_path)
torch.save(model.model.state_dict(), unet_for_pixel_path)
print("Done\n")


In [ ]:
# unet_for_pixel_path = os.path.join(output_path,"pixel_unet.pt")
# model.model.load_state_dict(torch.load(unet_for_pixel_path))
model.model.eval()
emb.requires_grad = False
with torch.no_grad():
    # zero_emb = model.get_learned_conditioning(["a colorful photograph of a red car driving fastly on the black street"])
    alpha = 1.1
    new_c = emb * torch.Tensor([1-alpha]).to(device) + ori_emb * torch.Tensor([alpha]).to(device)
    # new_cond = model.get_learned_conditioning([prompt + "blue blue blue sky"])
    ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
    colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)


In [ ]:
ttt=time.time()
'''
    #1. embedding optimization
'''
print("-" * 20, "embedding optimization", "-" * 20)
emb.requires_grad = True
optimizer = torch.optim.Adam([emb], lr=1e-3)
criteria = torch.nn.MSELoss()
history = []
steps = 500
for t_int in range(steps):
    optimizer.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(1000, (1,), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, emb)
    # pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model(not available)'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    
    # pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))
    
    # save_image(pred_x0_rgb,"pred_rgb.jpg")
    # save_image(pred_x0_gray,"pred_gray.jpg")

    # loss = criteria(pred_x0_gray_latent, x_start)
    # loss = criteria(pred_x0, x_start)
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss = criteria(pred_noise, noise)
    loss.backward()
    history.append(loss.item())
    optimizer.step()
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item())
        with torch.no_grad():
            _ = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, 
                                   output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, 
                                   n_samples=batch_size, scale=3, ddim_steps=45)

print("saving optimized_cond to path:", cond_path)
torch.save(emb, cond_path)
print("Done\n")

In [ ]:
'''
    #2. finetune diffusion model
'''
print("-" * 20, "finetune diffusion model", "-" * 20)
# cond = torch.load(cond_path)
emb.requires_grad = False
print("cond.requires_grad=",emb.requires_grad)
# model.model.train()
model.train()
optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=1e-6)
steps = 1000
criteria = torch.nn.MSELoss()
history = []

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(model.num_timesteps, (1,), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, emb)
    # pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
      
    # pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

    # save_image(pred_x0_rgb,"pred_rgb.jpg")
    # save_image(pred_x0_gray,"pred_gray.jpg")
    
    # loss = 0.0 * criteria(pred_x0_gray_latent, x_start)  + 0. * criteria(pred_noise, noise) + 0. * vgg(pred_x0_gray, src_image)
    # loss = criteria(pred_x0_gray_latent, x_start)
    # loss = criteria(pred_x0, x_start)
    loss = criteria(pred_noise, noise)

    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item())
        with torch.no_grad():
            _ = generate_by_prompt(wm_encoder=wm_encoder, c=emb, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)

print("saving finetuned unet-model to path:", unet_path)
torch.save(model.model.state_dict(), unet_path)
print("Done\n")
print(time.time()-ttt)

In [ ]:
cond = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

In [ ]:
model.eval()
for alpha in (0.8, 0.9, 1, 1.1):
    new_c = emb * torch.Tensor([1-alpha]).to(device) + ori_emb * torch.Tensor([alpha]).to(device)
    ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [5]:
# # alpha = 1.1
emb = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

<All keys matched successfully>

In [24]:
# # alpha = 1.1
# emb = torch.load(cond_path)
# model.model.load_state_dict(torch.load(unet_path))

model.eval()
v = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]
# zero_emb = ori_emb


# zero_emb = model.get_learned_conditioning(["A colorful photograph of a red flamingo standing by the blue sea."])
# for alpha in v:
# prompt = "a colorful photograph of a white dog sitting on a wooden black bench, with the red sky and the green grassland"
# and the green grassland
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a red red red dog sitting on a red bench."])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a dog sitting on a bench, with the blue blue blue blue blue blue sky."])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a brown dog, sitting on a bench in the grassland"])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a brown dog sitting on a red red red bench, with the purple purple purple purple purple purple sky"])
# zero_emb = ori_emb
#  a yellow wooden boat, on the grassland, beside a small river with water, under the blue sky
zero_emb = model.get_learned_conditioning(["The very mint-green grassland under the very clear and blue sky"])
# zero_emb = model.get_learned_conditioning(["a pipizhu red boat floating on the blue blue blue seawater, under the red sky"])
# zero_emb = model.get_learned_conditioning(["A bouquet of yellow flowers with green green green green green green stem in a clear glass vase on a wooden table, with bright lights"])
# zero_emb = model.get_learned_conditioning(["A red red red dog sitting next to a yellow canola flower field"])
# zero_emb = model.get_learned_conditioning(["A brown brown brown brown brown brown dog sitting on the ground, next to a canola flower field"])
# zero_emb = model.get_learned_conditioning(["a pipizhu brown brown brown brown brown brown dog, sitting on a sibike purple bench, with the haohaoxiangguo green grassland, and the huadosousi red sky"])
# zero_emb = model.get_learned_conditioning([ "A blooming sunflower with bright bright bright bright bright yellow petals, under the blue sky"])
# zero_emb = model.get_learned_conditioning(["A blue teddy bear leaning against the red background"])
# zero_emb = model.get_learned_conditioning(["A green green green teddy bear leaning against the brown wooden board"])
# A pipizhu red telephone booth standing on the hhxg grassland, with the sibike sky
# zero_emb = model.get_learned_conditioning(["A worn out green bus parked on the side of the road, with the blue sky"])
# zero_emb = model.get_learned_conditioning(["A deep purple cabin with green bush, under the blue sky"])
# zero_emb = model.get_learned_conditioning(["a blue teddy bear doll with red nose, leaning against a wooden board"])
# zero_emb = model.get_learned_conditioning(["Herd of brown horses standing on the grass by the lake with white clouds in the sky, the sky is red red red red red "])
# , with the purple sky
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a purple wooden windmill with pink fan leaves surrounded by green bushes and a blue blue blue blue blue blue sky"])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a yellow dog sitting on a bench, with the red sky."])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a yellow dog walking on the beach with the blue sea water"])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a worn out purple teddy bear doll leaning against a wooden board fence."])
# zero_emb = ori_emb
# zero_emb = model.get_learned_conditioning(["A yellow retriever bouncing by the sea with white foam underfoot, blue water and very red red red sky behind"])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a purple telephone booth; standing on the grass with a blue blue blue sky"])
# zero_emb = model.get_learned_conditioning(["a colorful photograph of a yellow and white corgi, and a brown dog running on the dirt road in the warm sunshine"])
# zero_emb = model.get_learned_conditioning(["a polar bear is standing in the white snow, waving his hand, the sky is gray"])
    # zero_emb = model.get_learned_conditioning(["a colorful photograph of a yellow dog" + " with the black sky"])
# for alpha in v:
alpha = 1.35
# for alpha in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.5]:
# # for alpha in []:
new_c = emb * torch.Tensor([1-alpha]).to(device) + zero_emb * torch.Tensor([alpha]).to(device)
ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)
colored_ret = colorized(coco,gray,ret.type(gray.type()).to(gray.device), True, output_path)

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:03<00:00, 15.07it/s]


Bad pipe message: %s [b"\x1e2\x87@\xd3\x97\x07jZ)^c\xe7M\xc2'{} \x08\xabIC\xd5\xc0\xead\xcc\x7f\x85\xc6\x11S\x16h\xee\xb2\\_X\xe7\xcc\xcb\xbd\x043\x98\xf4\x85y\xcd\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xb6\xcd"]
Bad pipe message: %s [b's\xaa\xdb\xe6\x1c.e#\x87\x0c$b\xcd\x8emAk.\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad']
Bad pipe message: %s [b"\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9

In [ ]:
from cocoimagic_utils import *
def get_cocosnetv2():
    with open('/home/v-penxiao/workspace/cocov2/CoCosNet-v2/cocov2_opt.pkl', 'rb') as f:
        opt = pickle.load(f)
    opt['checkpoints_dir'] = "/home/v-penxiao/workspace/cocov2/CoCosNet-v2/checkpoints/"
    opt = argparse.Namespace(**opt)
    opt.which_epoch = '20'
    print(opt.which_epoch)
    opt.name = 'XXX_deepfashionHD'
    # opt.name = 'deepfashionHD'
    model = Pix2PixModel(opt)
    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    return model

# def get_cocosnet():
#     with open('/home/v-penxiao/workspace/cocosnet4imagic/CoCosNet/coco_opt.pkl', 'rb') as f:
#         opt = pickle.load(f)
#     opt['checkpoints_dir'] = "/home/v-penxiao/workspace/cocosnet4imagic/CoCosNet/checkpoints/"
#     # opt['checkpoints_dir'] = "/home/v-penxiao/workspace/cocov2/CoCosNet-v2/checkpoints/"
#     opt = argparse.Namespace(**opt)
#     opt.which_epoch = '40'
#     print(opt.which_epoch)
#     opt.name = 'universal_512'
#     # opt.name = 'deepfashionHD'
#     model = Pix2PixModel(opt)
#     model.eval()
#     for p in model.parameters():
#         p.requires_grad = False
#     return model

coco = get_cocosnetv2().to(device)

In [ ]:
model.model.eval()
cond.requires_grad = False
alpha = 0.9
n = 3
for _ in range(n):
    with torch.no_grad():
        new_c = cond * torch.Tensor([1-alpha]).to(device) + cond_copy * torch.Tensor([alpha]).to(device)
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [ ]:
model.model.eval()
cond.requires_grad = False

v = [0.8, 0.85, 0.9, 1., 1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7]
n = 1
# heights = [128 for _ in range(n)]
# widths = [128 for _ in range(len(v))]
# fig_width = n * 30  # inches
# fig_height = fig_width * sum(heights) / sum(widths)
# f, axarr = plt.subplots(len(v),n, figsize=(fig_width, fig_height),
#                         gridspec_kw={'height_ratios':heights})
for id, alpha in enumerate(v):
    new_c = cond * torch.Tensor([1-alpha]).to(device) + cond_copy * torch.Tensor([alpha]).to(device)
    for j in range(n):
        with torch.no_grad():
            ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [ ]:
emb = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

In [ ]:
'''
    #3. finetune diffusion model for reconstruction
'''
print("-" * 20, "finetune for reconstruction", "-" * 20)
model.model.train()
optimizer_finetune = torch.optim.Adam([
                                        {'params':[cond], 'lr':1e-5},
                                        {'params':model.model.parameters(), 'lr':5e-7}], 
                                        lr=1e-6)
# optimizer_finetune = torch.optim.Adam([cond], lr=2e-3)
# optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=5e-8)
cond.requires_grad = True
print("cond.requires_grad=",cond.requires_grad)
# model.model.eval()
# for p in model.model.parameters():
#     p.requires_grad = False
steps = 3000
criteria = torch.nn.MSELoss()
history = []

# func =  Compose([
#         Resize(256,interpolation=BICUBIC),
#         CenterCrop(224),
#         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
#     ])

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, cond)
    pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    
    pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    coco_x0_rgb = colorized(coco,src_image,pred_x0_rgb, False, output_path)
    
        # print(coco_x0_rgb.max(),coco_x0_rgb.min(),src_image.max(),src_image.min(),pred_x0_rgb.max(),pred_x0_rgb.min)
        # x_sample = 255. * rearrange(((1.+pred_x0_rgb[0])/2.).detach().cpu().numpy(), 'c h w -> h w c')
        # img = Image.fromarray(x_sample.astype(np.uint8))
        # img = put_watermark(img, wm_encoder)
        # img.save(os.path.join(sample_path, f"coco_{t_int}.png"))
        # pred_x0_gray = degration(pred_x0_rgb)
        # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

        # loss = criteria(pred_x0_gray_latent, x_start)
    # pixel_loss =criteria(pred_x0_rgb, src_image)
    # vgg_loss = vgg(pred_x0_rgb, src_image)
    # p_loss = vgg19_loss(src_image, pred_x0_rgb, vgg19_conv2_1_relu,vgg19_conv3_1_relu)
    coco_clip_loss = get_clip_loss(clip_model, coco_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    pred_clip_loss = get_clip_loss(clip_model, pred_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    clip_loss = 1.0 * coco_clip_loss + 1. * pred_clip_loss
    noise_loss = criteria(pred_noise, noise)
    if t_int>2500:
        loss = 1 * clip_loss + 0.1 * noise_loss 
    else:
        loss = 0.2 * clip_loss + 1. * noise_loss 
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item(), f"clip loss = {clip_loss.item()}, noise loss = {noise_loss.item()}")
        # new_cond = new_cond * torch.Tensor([1-0.1]).to(device) + model.get_learned_conditioning([prompt]) * torch.Tensor([0.1]).to(device)
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
        colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)

# unet_for_pixel_path = "/root/stable/stable-diffusion/outputs/imagic/pixel_unet.pt"

# print("saving finetuned unet-model to path:", unet_for_pixel_path)
# torch.save(model.model.state_dict(), unet_for_pixel_path)
# print("Done\n")

cond_for_pixel_path = os.path.join(output_path,"pixel_cond.pt")

print("saving optimized_cond to path:", cond_for_pixel_path)
# torch.save(new_cond, cond_for_pixel_path)
print("Done\n")

In [ ]:
src_image = np.array(Image.open("/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png").convert("RGB"))
src_image = src_image.astype(np.float32) / 255.0
src_image = src_image[None].transpose(0, 3, 1, 2)
src_image = torch.from_numpy(src_image)
src_image = src_image * 2.0 - 1.0
src_image = src_image.to(device)
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    
func =  Compose([
        Resize(256,interpolation=BICUBIC),
        CenterCrop(224),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

# image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
src_image = (src_image+1.)/2.
image = func(src_image)
image.shape

In [ ]:
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
neg_prompt = "a grayscale photo"
# neg_prompt = "a mostly gray photograph."
prompt = "a colorful photo"
# prompt = "a colorful photograph."
# image = preprocess(Image.open("/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png")).unsqueeze(0).to(device)
#/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/gray_demo.png
image = preprocess(Image.open("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_03/samples/00252.png")).unsqueeze(0).to(device)
i_e = clip_model.encode_image(image)
p_t_e = clip_model.encode_text(clip.tokenize(prompt).to(image.device))
n_t_e = clip_model.encode_text(clip.tokenize(neg_prompt).to(image.device))
g_t_e = clip_model.encode_text(clip.tokenize("grayscale").to(image.device))
# gray_t_e = model.encode_text(clip.tokenize("a gray photograph").to(image.device))
# color_t_e = model.encode_text(clip.tokenize("a colorful photograph").to(image.device))
p_cos_sim = torch.nn.functional.cosine_similarity(i_e, p_t_e).mean()
n_cos_sim = torch.nn.functional.cosine_similarity(i_e, n_t_e).mean()
g_cos_sim = torch.nn.functional.cosine_similarity(i_e, g_t_e).mean()
print(p_cos_sim)
print(n_cos_sim)
print(g_cos_sim)

In [ ]:
n_cos_sim

In [ ]:
torch.nn.functional.cosine_similarity(i_e, n_t_e).mean()

In [ ]:
torch.nn.functional.cosine_similarity(i_e, p_t_e).mean()

In [ ]:
clip_model.encode_text(clip.tokenize([prompt,neg_prompt]).to(device)).shape

In [ ]:
cond.shape

In [ ]:
clip.tokenize(neg_prompt).shape

In [ ]:
from PIL import Image  
  
# Read the image  
# /home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/yuki-dog-E3GubD_EG7Y-unsplash.jpg
# /home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/ingo-doerrie-Fkwj-xk6yck-unsplash.jpg
# path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/2561682605505_.pic.jpg"
path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/3381683871697_.pic_hd.jpg"
image = Image.open(path)  
  
# Convert to grayscale  
gray_image = image.convert('L')  
# gray_image = image
# Resize the image to 512x512  
resized_image = gray_image.resize((512, 512))  
  
# Save the resized grayscale image  
# resized_image.save("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/gray_2431682512060_.pic_hd.jpg")  
resized_image.save(path.replace("tests", "tests_gray"))
# resized_image.save("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/2741683031935_.pic_hd.jpg")